# Backtest Example

In this notebook I will demonstrate a dynamic scan over several data feeds.
To simulate a scan I attached each stock with months in which it is allowed to be bought, as if it was shown in the scans in those months.
NVDA (1,7)
AAPL (2,8)
MSFT (3,9)
GOOG (4,10)
TSLA (5,11)
AMZN (6,12)

Buying criteria:
- Current month is an allowed month
- 10 ema and 21 ema sloping upward
- 10 ema above 21 ema

Selling criteria:
- 10 ema and 21 ema sloping down
- 10 ema below 21 ema

In [5]:
from datetime import datetime
import backtrader as bt
from backtrader.analyzers import AnnualReturn, TradeAnalyzer
import yfinance as yf
from stats.tradeLog import TradeLog
from stats.strategySummery import build_summary_report
from stats.exporter import export_csv


class SmaCross(bt.Strategy):
    params = dict(
        period_fast=10,
        period_slow=21
    )

    def __init__(self):
        self.sma_pairs = []

        i = 1
        for data in self.datas:
            sma10 = bt.ind.SMA(data, period=self.p.period_fast)
            sma21 = bt.ind.SMA(data, period=self.p.period_slow)
            months_to_trade = [i, i+6]
            self.sma_pairs.append((data, sma10, sma21, months_to_trade))
            i = i + 1

    def next(self):
        for data, sma10, sma21, months_to_trade in self.sma_pairs:
            if not len(data) > self.p.period_slow:
                continue

            current_date = self.data.datetime.date(0)
            is_positioned = self.getposition(data)

            sma10_above_sma21 = sma10[0] > sma21[0]
            sma10_sloping_up = sma10[0] > sma10[-1]
            sma21_sloping_up = sma21[0] > sma21[-1]

            if not is_positioned and current_date.month in months_to_trade:
                if sma10_above_sma21 and sma10_sloping_up and sma21_sloping_up:
                    self.buy(data=data, size=10)

            elif is_positioned:
                if not sma10_above_sma21 and not sma10_sloping_up and not sma21_sloping_up:
                    self.close(data=data)


stocks_pool = ["NVDA", "AAPL", "MSFT", "GOOG", "TSLA", "AMZN"]

cerebro = bt.Cerebro()
cerebro.addsizer(bt.sizers.PercentSizer)
cerebro.broker.setcash(100000)

for stock in stocks_pool:
    stocks_data = yf.download(stock, start=datetime(2017, 1, 1), end=datetime(2024, 12, 31))
    stocks_data.columns = ["Open", "High", "Low", "Close", "Volume"] # Compatability with backtrader pandasData format
    stock_datafeed = bt.feeds.PandasData(dataname=stocks_data, name=stock)
    cerebro.adddata(stock_datafeed)

cerebro.addstrategy(SmaCross)
cerebro.addanalyzer(TradeLog, _name='trade_log')
sma_cross_strategy = cerebro.run()[0]

trade_table = sma_cross_strategy.analyzers.trade_log.get_analysis()
strategy_name = sma_cross_strategy.__class__.__name__
report = build_summary_report(sma_cross_strategy)
log_path = export_csv(trade_table, strategy_name)
summary_csv_path = export_csv(report, strategy_name)



cerebro.plot()
sma_cross_strategy.analyzers[0].get_analysis()



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


<IPython.core.display.Javascript object>

[{'trade_id': 5,
  'data_name': 'AMZN',
  'side': 'long',
  'entry_time': '2017-06-02T00:00:00',
  'exit_time': '2017-06-13T00:00:00',
  'entry_price': 50.33649826049805,
  'exit_price': 34.20589116968116,
  'size': 10.0,
  'position_value_at_entry': 503.36498260498047,
  'position_fraction_of_equity': 0.005024495796943983,
  'pnl_abs': 43.22338104248047,
  'pnl_pct': 0.08586886759343836,
  'holding_period_days': 11.0},
 {'trade_id': 6,
  'data_name': 'GOOG',
  'side': 'short',
  'entry_time': '2017-06-28T00:00:00',
  'exit_time': '2017-06-28T00:00:00',
  'entry_price': 42.84479904174805,
  'exit_price': 46.134291100136075,
  'size': 0.0,
  'position_value_at_entry': 0.0,
  'position_fraction_of_equity': 0.0,
  'pnl_abs': 38.60088348388672,
  'pnl_pct': 0.0,
  'holding_period_days': 0.0},
 {'trade_id': 7,
  'data_name': 'MSFT',
  'side': 'short',
  'entry_time': '2017-06-29T00:00:00',
  'exit_time': '2017-06-29T00:00:00',
  'entry_price': 58.152130126953125,
  'exit_price': 63.15518754